In [2]:
# 이미지 별 x, y 좌표 추출
def extract_coordinates(dataframe):
    coordinates = {}
    for image_name in dataframe['image_name'].unique():
        # 해당 이미지의 데이터 필터링
        image_data = dataframe[dataframe['image_name'] == image_name]
        # x, y 좌표 배열 생성
        x_coords = image_data['x'].tolist()
        y_coords = image_data['y'].tolist()
        # 딕셔너리에 저장
        coordinates[image_name] = {'x': x_coords, 'y': y_coords}
    return coordinates

In [3]:
import numpy as np
import pandas as pd

neutral_df = pd.read_csv("../output/neutral.csv")
angry_df = pd.read_csv("../output/angry.csv")
happy_df = pd.read_csv("../output/happy.csv")
neutral_df['label'] = 0
happy_df['label'] = 1
angry_df['label'] = 2

face_metadata_df = pd.concat([neutral_df, angry_df, happy_df], ignore_index=True)
face_metadata_df = face_metadata_df.sort_values(by="image_name")

In [4]:
face_metadata_df.head(5)

,image_name,face_id,point_id,x,y,label
0,C0000.jpg,0,0,152,106,0
36,C0000.jpg,0,36,159,103,0
37,C0000.jpg,0,37,161,101,0
38,C0000.jpg,0,38,165,101,0
39,C0000.jpg,0,39,169,104,0


In [5]:
x_face_metadata_df = face_metadata_df.drop('label', axis=1)
y_face_metadata_df = face_metadata_df[['image_name', 'label']]

In [6]:
x_face_metadata_df.head(5)

,image_name,face_id,point_id,x,y
0,C0000.jpg,0,0,152,106
36,C0000.jpg,0,36,159,103
37,C0000.jpg,0,37,161,101
38,C0000.jpg,0,38,165,101
39,C0000.jpg,0,39,169,104


In [7]:
y_face_metadata_df.head(5)

,image_name,label
0,C0000.jpg,0
36,C0000.jpg,0
37,C0000.jpg,0
38,C0000.jpg,0
39,C0000.jpg,0


In [8]:
from sklearn.model_selection import train_test_split

def convert_to_X_train(target_df: pd.DataFrame) -> np.ndarray:
    return np.array([
        group[["x", "y"]].values
        for _, group in target_df.groupby("image_name")
    ])
    
x_train_data = convert_to_X_train(x_face_metadata_df)

In [9]:
def convert_to_Y_train(target_df: pd.DataFrame):
    result = target_df.drop_duplicates('image_name').copy()
    result['neutral'] = result['label'].map({ 0: 1, 1: 0, 2: 0})
    result['happy'] = result['label'].map({ 0: 0, 1: 1, 2: 0 })
    result['angry'] = result['label'].map({ 0: 0, 1: 0, 2: 1 })                                           
    
    return np.array(result.drop(['image_name', 'label'], axis=1))

y_train_data = convert_to_Y_train(y_face_metadata_df)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train_data,
                                                    y_train_data,
                                                    test_size=0.2,
                                                    random_state=11)

In [11]:
print(X_train.shape[0])
print(Y_train.shape[0])

66664
66664


In [14]:
%pip install tensorflow

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, BatchNormalization, GlobalAveragePooling1D
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau


  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/390.2 MB ? eta -:--:--
   ---------------------------------------- 4.5/390.2 MB 20.7 MB/s eta 0:00:19
   - --------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [15]:
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(68, 2)),
    BatchNormalization(),  # 모델 안정화
    Dense(32 , activation='relu'),
    Dense(32 , activation='relu'),
    Dense(64 , activation='relu'),
    Dense(128, activation='relu'), 
    Dense(256, activation='relu'), Dropout(0.5),
    Dense(256, activation='relu'), Dropout(0.5),  
    Dense(128, activation='relu'),
    Dense(64 , activation='relu'),
    Dense(32 , activation='relu'),
    Dense(32 , activation='relu'),
    
    GlobalAveragePooling1D(),
    Dense(3, activation='softmax') # 출력층
])

c:\Users\kevin\anaconda3\envs\facedetect\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
from model_analyzer import NEpochCallback, NBatchCallback

In [17]:
batch_callback = NBatchCallback(n=10)
epoch_callback = NEpochCallback(n= 2, filepath="./weights/")

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(X_train, Y_train, epochs=10, batch_size=128, callbacks=[batch_callback, epoch_callback])

Epoch 1/10
  9/521 ━━━━━━━━━━━━━━━━━━━━ 1:04 126ms/step - accuracy: 0.3517 - loss: 1.0976Batch 10: logs={'accuracy': 0.35859376192092896, 'loss': 1.0978196859359741}
 19/521 ━━━━━━━━━━━━━━━━━━━━ 1:06 132ms/step - accuracy: 0.3545 - loss: 1.0978Batch 20: logs={'accuracy': 0.34648436307907104, 'loss': 1.0987248420715332}
 29/521 ━━━━━━━━━━━━━━━━━━━━ 1:07 137ms/step - accuracy: 0.3498 - loss: 1.0981Batch 30: logs={'accuracy': 0.33645832538604736, 'loss': 1.0986422300338745}
 39/521 ━━━━━━━━━━━━━━━━━━━━ 1:03 131ms/step - accuracy: 0.3467 - loss: 1.0983Batch 40: logs={'accuracy': 0.33808594942092896, 'loss': 1.0984891653060913}
 49/521 ━━━━━━━━━━━━━━━━━━━━ 59s 127ms/step - accuracy: 0.3453 - loss: 1.0983 Batch 50: logs={'accuracy': 0.33937498927116394, 'loss': 1.0983706712722778}
 59/521 ━━━━━━━━━━━━━━━━━━━━ 57s 124ms/step - accuracy: 0.3443 - loss: 1.0983Batch 60: logs={'accuracy': 0.33776041865348816, 'loss': 1.098501443862915}
 69/521 ━━━━━━━━━━━━━━━━━━━━ 54s 121ms/step - accuracy: 0.343

In [19]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score

In [20]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(Y_test, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

521/521 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step
Accuracy: 0.5663026521060842, Precision: 0.6355673531583363, Recall: 0.5663026521060842, F1 Score: 0.5164191257756707
